In [18]:
import pandas as pd
import numpy as np
import re
from pprint import pprint
import collections
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers
import json
from nltk.corpus import stopwords

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import os

In [35]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['http', 'bully', 'bullying', 'rt'])
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [32]:
df = pd.read_csv('../text_twitter_type_raw.csv')
print(df.Type.unique())
types=['sexism', 'racism']

df = df.loc[df['Type'].isin(types)]
print(df.Type.unique())
print(df)


['none' 'sexism' 'racism']
['sexism' 'racism']
                                                    Text    Type     Label
2      RT @username Call me sexist, but when I go to ...  sexism  positive
3      @username Wrong, ISIS follows the example of M...  racism  positive
6      RT @username @username nooo not sexist but mos...  sexism  positive
9      RT @username There's something wrong when a gi...  sexism  positive
11     @username @username @username @username A good...  racism  positive
...                                                  ...     ...       ...
16832  RT @username Just don't find Woman comedians f...  sexism  positive
16834  RT @username I'm not sexist but I hate all fem...  sexism  positive
16836  @username Yes, all that Islam is making ISIS s...  racism  positive
16841  yeah so annoying how they're doing their jobs ...  sexism  positive
16844  @username An enemy of @username is good enough...  sexism  positive

[5347 rows x 3 columns]


In [33]:
"""remove_rt = lambda x: re.sub('RT @\w+: ', "", x)
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",x)
df.Text = df.Text.map(remove_rt).map(rt)

print(df.Text)"""

# Convert to list
data = df.Text.values.tolist()

data = [re.sub('RT @\w+: ', "",sent) for sent in data]

#remove links
data = [re.sub(r"http\S+", "", sent) for sent in data]


# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['RT Call me sexist, but when I go to an auto place, Id rather talk to a guy']


In [60]:

from nltk.internals import config_java

os.environ['JAVAHOME'] = "C:/Program Files/Java/jdk1.8.0_161/bin/java.exe"
config_java('C:/Program Files/Java/jdk1.8.0_161/bin/java.exe')


english_nertagger = StanfordNERTagger('C:/Users/erajkovic/Anaconda3/Lib/site-packages/stanford-ner-2015-04-20/classifiers/english.conll.4class.distsim.crf.ser.gz', 'C:/Users/erajkovic/Anaconda3/Lib/site-packages/stanford-ner-2015-04-20/stanford-ner.jar')

In [14]:
from nltk.tag.stanford import StanfordNERTagger

#Set environmental variables programmatically.
#Set the classpath to the path where the jar file is located
os.environ['CLASSPATH'] = "C:/Users/erajkovic/Documents/APT/projekt/stanford-ner-2015-04-20/stanford-ner.jar"
#Set the Stanford models to the path where the models are stored
os.environ['STANFORD_MODELS'] = 'C:/Users/erajkovic/Documents/APT/projekt/stanford-corenlp-caseless-2015-04-20-models/edu/stanford/nlp/models/ner'
#Set the java jdk path

java_path = "C:/Program Files/Java/jdk1.8.0_161/bin/java.exe"
os.environ['JAVAHOME'] = java_path


#Set the path to the model that you would like to use
stanford_classifier  =  'C:/Users/erajkovic/Documents/APT/projekt/stanford-corenlp-caseless-2015-04-20-models/edu/stanford/nlp/models/ner/english.all.3class.caseless.distsim.crf.ser.gz'

#Build NER tagger object
st = StanfordNERTagger(stanford_classifier, "C:/Users/erajkovic/Documents/APT/projekt/stanford-ner-2015-04-20/stanford-ner.jar")

#A sample text for NER tagging
text = 'srinivas ramanujan went to the united kingdom. There he studied at cambridge university.'

#Tag the sentence and print output
tagged = st.tag(str(text).split())
print(tagged)
print(str(text).split())


[('srinivas', 'PERSON'), ('ramanujan', 'PERSON'), ('went', 'O'), ('to', 'O'), ('the', 'O'), ('united', 'LOCATION'), ('kingdom.', 'LOCATION'), ('There', 'O'), ('he', 'O'), ('studied', 'O'), ('at', 'O'), ('cambridge', 'LOCATION'), ('university.', 'O')]
['srinivas', 'ramanujan', 'went', 'to', 'the', 'united', 'kingdom.', 'There', 'he', 'studied', 'at', 'cambridge', 'university.']


In [36]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=False))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['rt', 'call', 'me', 'sexist', 'but', 'when', 'go', 'to', 'an', 'auto', 'place', 'id', 'rather', 'talk', 'to', 'guy']]


In [15]:
print(st.tag(data_words[:1][0]))

[('rt', 'O'), ('call', 'O'), ('me', 'O'), ('sexist', 'O'), ('but', 'O'), ('when', 'O'), ('go', 'O'), ('to', 'O'), ('an', 'O'), ('auto', 'O'), ('place', 'O'), ('id', 'O'), ('rather', 'O'), ('talk', 'O'), ('to', 'O'), ('guy', 'O')]


In [ ]:
def check_ner_tag(text, word):
    tagged = st.tag(text)
    for w, tag in tagged:
        if w == word:
            if tag == 'PERSON':
                return True
            else:
                return False
    return False

def remove_ner_person(texts):
    return [[word for word in doc if not check_ner_tag(doc, word)] for doc in texts]

data_words = remove_ner_person(data_words)

data_words

In [37]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

C:\Users\erajkovic\Anaconda3\envs\tensorflow_env\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['rt', 'call', 'me', 'sexist', 'but', 'when', 'go', 'to', 'an', 'auto', 'place', 'id', 'rather', 'talk', 'to', 'guy']


In [38]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def remove_no_dict_words(texts):
    return [[word for word in doc if word in glove_dict] for doc in texts]
    
GLOVE_DIM=100
glove_file = 'C:/Users/erajkovic/Downloads/glove.twitter.27B/glove.twitter.27B.100d.txt'
glove_dict = []
NB_WORDS = 4525   # duljina file-a vocab
glove = open(glove_file, encoding="utf8")
for line in glove:
    values = line.split()
    word = values[0]
    glove_dict.append(word)
glove.close()
print(glove_dict[:3])
    

['<user>', '.', ':']


In [39]:
# Remove Stop Words
data_words_nodict = remove_no_dict_words(data_words)
data_words_nostops = remove_stopwords(data_words_nodict)
print(data_words_nostops)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized = remove_stopwords(data_lemmatized)


print(data_lemmatized[:1])


[['call', 'sexist', 'go', 'auto', 'place', 'id', 'rather', 'talk', 'guy'], ['wrong', 'isis', 'follows', 'example', 'mohammed', 'quran', 'exactly'], ['nooo', 'sexist', 'women', 'bad', 'drivers'], ['theres', 'something', 'wrong', 'girl', 'wins', 'wayne', 'rooney', 'street', 'striker'], ['good', 'muslim', 'good', 'despite', 'bad', 'religion'], ['yeah', 'called', 'caring', 'human', 'life', 'idiot', 'something', 'genocidal', 'daesh', 'wouldnt', 'understand'], ['literally', 'site', 'today'], ['according', 'holly', 'geordie', 'shore', 'sassy', 'halfway', 'slut', 'classy', 'mkr', 'mkr'], ['aww', 'looking', 'forward', 'ash', 'camilla', 'butting', 'heads', 'promo', 'girls', 'drama', 'wouldve', 'great', 'mkr'], ['muslim', 'brain', 'dead', 'repeat', 'others', 'said', 'million', 'times'], ['im', 'sexist', 'dont', 'believe', 'idiosyncrasies', 'certain', 'females', 'males'], ['guys', 'sell', 'drugs', 'get', 'girls', 'guys', 'degrees', 'says', 'women', 'men'], ['im', 'sure', 'tweet', 'used', 'court', 

[['call', 'sexist', 'go', 'auto', 'place', 'rather', 'talk', 'guy']]


In [41]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
tfidf = TfidfModel(corpus)
corpus = tfidf[corpus]

# View
print(corpus[:1])
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus]

[[('auto', 0.5943177881346746),
  ('call', 0.22600343883966473),
  ('go', 0.24344915929287916),
  ('guy', 0.3233879036355336),
  ('place', 0.39555842400254493),
  ('rather', 0.4040937207453535),
  ('sexist', 0.12817429486848395),
  ('talk', 0.30931332197453887)],
 [('exactly', 0.41679376444951766),
  ('example', 0.4486112661021463),
  ('follow', 0.3855131102245489),
  ('isis', 0.32059329054483277),
  ('mohamme', 0.30921407937084533),
  ('quran', 0.33363924885281254),
  ('wrong', 0.40829153614621033)],
 [('sexist', 0.15513951985278698),
  ('bad', 0.3853828031062986),
  ('driver', 0.42591998580573337),
  ('nooo', 0.7825366884601603),
  ('woman', 0.18341298033353137)],
 [('wrong', 0.2574794349685156),
  ('girl', 0.14786519253399927),
  ('rooney', 0.482914306926467),
  ('street', 0.34801930598570585),
  ('striker', 0.482914306926467),
  ('wayne', 0.482914306926467),
  ('win', 0.30183456489208654)],
 [('bad', 0.44111946459540285),
  ('good', 0.7788720771234389),
  ('muslim', 0.2354352226959

In [44]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=2, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [45]:
# Print the Keyword in the n topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.010*"islam" + 0.007*"know" + 0.006*"feminist" + 0.006*"woman" + '
  '0.005*"go" + 0.004*"say" + 0.004*"see" + 0.004*"religion" + 0.004*"well" + '
  '0.003*"hate"'),
 (1,
  '0.013*"mkr" + 0.010*"sexist" + 0.008*"kat" + 0.008*"muslim" + 0.007*"woman" '
  '+ 0.006*"man" + 0.005*"think" + 0.005*"girl" + 0.005*"call" + '
  '0.005*"female"')]


In [46]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.491750091648784

Coherence Score:  0.25811809776060757


In [48]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.023682  0.0       1        1  53.659902
0     -0.023682  0.0       2        1  46.340098, topic_info=         Term       Freq      Total Category  logprob  loglift
105       kat  59.000000  59.000000  Default  30.0000  30.0000
43        mkr  97.000000  97.000000  Default  29.0000  29.0000
159     islam  75.000000  75.000000  Default  28.0000  28.0000
142      know  42.000000  42.000000  Default  27.0000  27.0000
177  feminist  40.000000  40.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
367  feminism  13.135022  22.528595   Topic2  -6.1275   0.2297
19       girl  17.569113  51.028956   Topic2  -5.8366  -0.2971
6      sexist  17.879801  88.702086   Topic2  -5.8191  -0.8324
540    stupid  12.406773  23.112828   Topic2  -6.1845   0.1470
26     muslim  11.798420  67.108963   Topic2  -6.2348  -0.9692

[128 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1624      1  0.271947    ago
1624      2  0.725192    ago
213       1  0.896449    amp
213       2  0.128064    amp
387       1  0.930066  andre
...     ...       ...    ...
300       2  0.102449    way
381       1  0.214482   well
381       2  0.786434   well
18        1  0.588453  woman
18        2  0.408314  woman

[180 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [ ]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
os.environ['MALLET_HOME'] = 'C:/Users/erajkovic/Downloads/mallet-2.0.8/mallet-2.0.8'
mallet_path = "C:/Users/erajkovic/Downloads/mallet-2.0.8/mallet-2.0.8/bin/mallet" # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=2, id2word=id2word)

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:

model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=24, step=3)

In [ ]:
# Show graph
limit=24; start=2; step=3;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topic =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[0]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [108]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)


C:\Users\erajkovic\Anaconda3\envs\tensorflow_env\lib\site-packages\smart_open\smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,7.0,0.1046,"love, anti, fight, send, post, form, kill, fac...","mr. walt, stop being a bully."
1,1,2.0,0.1129,"big, watch, ass, stand, feel, attack, thing, f...",totally. on one hand the retaliatory nature is...
2,2,1.0,0.1190,"people, make, friend, give, end, problem, haha...","yes on thur, she said she might, well bully in..."
3,3,6.0,0.1149,"shit, back, mom, abuse, beat, day, cry, ugly, ...",nigga used to be a real herbalist bully idk wh...
4,4,0.0,0.1046,"video, cyber, real, read, good, world, check, ...",bobby needs threw out that is disgusting #swea...
5,5,4.0,0.1066,"stop, hate, put, remember, stuff, bedroom, foo...",Please help to stop workplace bullying
6,6,7.0,0.1372,"love, anti, fight, send, post, form, kill, fac...","BETTER ANTI-BULLYING AD SLOGANS: Hey, Bullies...."
7,7,2.0,0.1087,"big, watch, ass, stand, feel, attack, thing, f...",vc sofres bullying
8,8,9.0,0.1128,"lol, life, man, victim, person, wanna, tweet, ...",[110830/TRANS] Strong Heart - Donghae Being Bu...
9,9,0.0,0.0909,"video, cyber, real, read, good, world, check, ...",is a bully


In [109]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.1424,"video, cyber, real, read, good, world, check, ...",Victory Fitness (www.vicfit.net) will be hosti...
1,1.0,0.2258,"people, make, friend, give, end, problem, haha...",#Itreallymakesmemad when people make fun of pe...
2,2.0,0.1655,"big, watch, ass, stand, feel, attack, thing, f...",first I aint big for no reason. Second I dont ...
3,3.0,0.1675,"school, student, rebecca_black, high, great, l...",Rebecca Black Quits Middle School After Bullyi...
4,4.0,0.1526,"stop, hate, put, remember, stuff, bedroom, foo...","""Courage is fire, and bullying is smoke"". Benj..."
5,5.0,0.1747,"time, child, boy, word, bring, play, act, wit,...",Kids bullying is an ADULT problem. Learn more ...
6,6.0,0.1406,"shit, back, mom, abuse, beat, day, cry, ugly, ...","#OnlyRealJamaicans eat ""bully beef"" outside fi..."
7,7.0,0.1528,"love, anti, fight, send, post, form, kill, fac...",Michelle ( Heartless Bitch) Bachman WONT Sign ...
8,8.0,0.1749,"kid, twitter, nice, good, brother, dude, game,...",#Bahrain :A Smart Little Kid #Saudi & #UAE :Th...
9,9.0,0.1480,"lol, life, man, victim, person, wanna, tweet, ...","I believe in Free speach, I dont believe in of..."


In [111]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
print(topic_counts)

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

0.0     91
2.0     76
3.0     64
9.0     62
8.0     60
1.0     59
4.0     58
6.0     52
7.0     47
5.0     46
10.0    42
Name: Dominant_Topic, dtype: int64


,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0.0,7.0,"love, anti, fight, send, post, form, kill, fac...",91.0,0.1385
1.0,2.0,"big, watch, ass, stand, feel, attack, thing, f...",59.0,0.0898
2.0,1.0,"people, make, friend, give, end, problem, haha...",76.0,0.1157
3.0,6.0,"shit, back, mom, abuse, beat, day, cry, ugly, ...",64.0,0.0974
4.0,0.0,"video, cyber, real, read, good, world, check, ...",58.0,0.0883
...,...,...,...,...
652.0,2.0,"big, watch, ass, stand, feel, attack, thing, f...",NaN,NaN
653.0,5.0,"time, child, boy, word, bring, play, act, wit,...",NaN,NaN
654.0,4.0,"stop, hate, put, remember, stuff, bedroom, foo...",NaN,NaN
655.0,4.0,"stop, hate, put, remember, stuff, bedroom, foo...",NaN,NaN
